In [13]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from SimilarityHelpers import get_cos_similarity_score
from sentence_transformers import SentenceTransformer
from enum import Enum
from typing import List, Dict, Optional
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
random_seed = 42

## Load Data

Steps:
- Load the dataset(s)
- (Optional) Do any splitting if needed
- (Optional) Filter any low quality input/output pairs
- Rename all article/text/content/body columns to "input"
- Rename all summary/label/output columns to "summary" 


Notes:
- Renaming is done so its easier during inference time to make model calls
- Some datasets are too large to load reliably so we split and use only a portion of the data

In [10]:
samsum = load_dataset("Samsung/samsum", trust_remote_code=True)['test']
samsum

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})

In [11]:
webis = load_dataset("webis/tldr-17", trust_remote_code=True)
webis

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary'],
        num_rows: 3848330
    })
})

In [21]:
# Take a random 10% of Webis
webis_sel = webis['train'].shuffle(seed=random_seed).train_test_split(test_size=0.01)
webis_test = webis_sel['test']
webis_test

Dataset({
    features: ['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary'],
    num_rows: 38484
})

In [22]:
# Normalize the dataset input/output columns so it is input for the model input and summary for the gold label
webis_test = webis_test.rename_columns({"content": "input"})
webis_test

Dataset({
    features: ['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'input', 'summary'],
    num_rows: 38484
})

In [23]:
samsum = samsum.rename_columns({"dialogue": "input"})
samsum

Dataset({
    features: ['id', 'input', 'summary'],
    num_rows: 819
})

## Helper Methods and Classes

In [27]:
class TaskPrefix(Enum):
    """
    """
    INSTRUCTION_AT_TOP = "Summarize the following text: "

    def __str__(self):
        return self.value

In [25]:
class Model(Enum):
    """
    Models supported by HuggingFace we can use for testing.
    We picked models that come in multiple sizes.
    - HuggingFaceTB/SmolLM2-135M
    """
    SMOL_LM2_135M = "HuggingFaceTB/SmolLM2-135M"

    def __str__(self):
        return self.value

In [34]:
class HuggingFaceInference:
    def __init__(self, model: Model):
        self.tokenizer = AutoTokenizer.from_pretrained(model.value)
        self.model = AutoModelForCausalLM.from_pretrained(model.value)

        # Add a padding token to the tokenizer if it doesn't exist
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({"pad_token": "<pad>"})

    def predict(self,  prompt: str, options: Optional[Dict] = None) -> str:
        """
        Predicts the output of a model given a prompt using HuggingFace transformers.
        """
        if options is None:
            options = {}
            
        try:
            # Tokenize and generate
            inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            outputs = self.model.generate(
                **inputs,
                temperature=options.get("temperature", 0.001),
                top_p=options.get("top_p", 0.9),
                do_sample=True
            )
            
            return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        except Exception as e:
            print(f"Error during prediction: {e}")
            return "None"

    def predict_batch(self, prompts: List[str], options: Optional[Dict] = {}) -> List[str]:
        """
        Batch prediction using HuggingFace transformers.
        """
        return [self.predict(prompt, options) for prompt in prompts]

In [35]:
def create_model_results_row():
    return {
        "Model": None,
        "Task_Prefix": None,
        "Dataset_Name": None,
        "Model_Responses": None,
        "Gold_Labels": None,
    }


## Collect LLM Responses

In [37]:
MODEL_EVALS = [
    {
        "Model": Model.SMOL_LM2_135M,
    }
]

In [38]:
DATASET_EVALS = [
    {
        "Name": "Samsum",
        "Dataset": samsum,
    },
    {
        "Name": "Webis",
        "Dataset": webis_test,
    }
]


In [39]:
model_results = {}

In [ ]:
for dataset in DATASET_EVALS:
    for model in MODEL_EVALS:
        print(f"Evaluating {model['Model']} on {dataset['Name']}")

        # Load the model
        model_obj = HuggingFaceInference(model['Model'])

        # Get the prompts
        inputs = dataset['Dataset']['input']

        # TODO: We should maybe vary this somehow
        # Map the inputs with the task prefix
        prompts = [TaskPrefix.INSTRUCTION_AT_TOP.value + "\n" + input for input in inputs]

        # Get the gold labels
        summaries = dataset['Dataset']['summary']

        # Get the model responses
        model_responses = model_obj.predict_batch(prompts, model['Options'] if model['Options'] else {})

        # Save the results
        model_results_row = create_model_results_row()
        model_results_row['Model'] = model['Model']
        model_results_row['Task_Prefix'] = TaskPrefix.INSTRUCTION_AT_TOP.value
        model_results_row['Dataset_Name'] = dataset['Name']
        model_results_row['Model_Responses'] = model_responses
        model_results_row['Gold_Labels'] = summaries

        model_results[f"{model['Model']}_{dataset['Name']}"] = model_results_row


Evaluating HuggingFaceTB/SmolLM2-135M on Samsum


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for o

Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, ot

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for o

Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, ot

...

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for o

Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, ot

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.
Error during prediction: `temperature` (=0) has to be a strictly positive float, ot

KeyboardInterrupt: 

In [33]:
model_results

{'HuggingFaceTB/SmolLM2-135M_Samsum': {'Model': <Model.SMOL_LM2_135M: 'HuggingFaceTB/SmolLM2-135M'>,
  'Task_Prefix': 'Summarize the following text: ',
  'Dataset_Name': 'Samsum',
  'Model_Responses': ['None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None',
   'None'

## Evaluate LLM Responses for contamination